<a href="https://colab.research.google.com/github/Kominkin/Kominkin.global.h/blob/master/Untitle16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.io.json import json_normalize

sns.set()

#Drive上のデータセットのパスを指定
import os
path = '/content/drive/MyDrive/comp_国勢調査からの収入予測/'
#os.listdirでパスで指定してディレクトリ上のファイル一覧を表示。
os.listdir(path)

#データの読み込み
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+"test.csv")
sample_submit = pd.read_csv(path+"sample_submit.csv",header=None)

#display(train.head())
#display(test.head())

#native-country, race, fnlwgt　の列を削除
train.drop(columns=['native-country', 'race', 'fnlwgt','relationship'], inplace=True)
test.drop(columns=['native-country', 'race', 'fnlwgt','relationship'], inplace=True)
display(train.head())
display(test.head())

train['marital-status'].value_counts()

#MartialとEducation-numの要素を置換
train = train.replace({'Never-married':'Not-married','Divorced':'Not-married','Separated':'Not-married','Widowed':'Not-married'})
train['marital-status'].value_counts()
test = test.replace({'Never-married':'Not-married','Divorced':'Not-married','Separated':'Not-married','Widowed':'Not-married'})
train = train.replace({3:8,4:8,5:8,6:8,7:8})
test = test.replace({3:8,4:8,5:8,6:8,7:8})                 

#object or category がダミー化されるので
#train.info()

train = pd.get_dummies(train)
test = pd.get_dummies(test)

display(train.head(3))
display(test.head(3))

features = train.columns.values.tolist()

#Yをリストから削除しておく。
features.remove('index')
features.remove('Y')
display(features)

#testにはないユニーク値のダミーを追加しておく。
test['education_1st-4th'] = 0
display(test)

# 学習データの説明変数(変数に代入されているので変数を指定すればよい
X_train = train[features]
#目的変数
y_train = train['Y']

X_test = test[features]

# LightGBM
import lightgbm as lgb

lgb_clf = lgb.LGBMClassifier(max_depth=3)
lgb_clf.fit(X_train, y_train)

y_pred = lgb_clf.predict(X_test)
print(y_pred)

#sample_submissionの右側のカラムに予測値を代入する。
sample_submit.iloc[:, -1] = y_pred
sample_submit.head()

display(sample_submit[1].value_counts())

,index,age,workclass,education,education-num,marital-status,occupation,sex,Y
0,322,21,Private,Some-college,10,Divorced,Adm-clerical,Female,0
1,11968,29,Private,HS-grad,9,Divorced,Adm-clerical,Female,0
2,10868,19,Private,Bachelors,13,Never-married,Prof-specialty,Female,0
3,3394,17,Private,HS-grad,9,Never-married,Craft-repair,Male,0
4,15993,47,Private,Some-college,10,Married-civ-spouse,Craft-repair,Male,0


,index,age,workclass,education,education-num,marital-status,occupation,sex
0,3873,17,Local-gov,Masters,14,Married-civ-spouse,Prof-specialty,Male
1,3625,23,Private,Some-college,10,Married-civ-spouse,Sales,Male
2,3028,19,Private,11th,7,Never-married,Handlers-cleaners,Female
3,13814,30,State-gov,HS-grad,9,Never-married,Protective-serv,Female
4,15398,60,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Male


,index,age,education-num,Y,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married-civ-spouse,marital-status_Not-married,occupation_?,occupation_Adm-clerical,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,sex_Female,sex_Male
0,322,21,10,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,11968,29,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,10868,19,13,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


,index,age,education-num,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,education_10th,education_11th,education_12th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married-civ-spouse,marital-status_Not-married,occupation_?,occupation_Adm-clerical,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,sex_Female,sex_Male
0,3873,17,14,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,3625,23,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,3028,19,8,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


['age',
 'education-num',
 'workclass_?',
 'workclass_Federal-gov',
 'workclass_Local-gov',
 'workclass_Private',
 'workclass_Self-emp-inc',
 'workclass_Self-emp-not-inc',
 'workclass_State-gov',
 'education_10th',
 'education_11th',
 'education_12th',
 'education_1st-4th',
 'education_5th-6th',
 'education_7th-8th',
 'education_9th',
 'education_Assoc-acdm',
 'education_Assoc-voc',
 'education_Bachelors',
 'education_HS-grad',
 'education_Masters',
 'education_Prof-school',
 'education_Some-college',
 'marital-status_Married-civ-spouse',
 'marital-status_Not-married',
 'occupation_?',
 'occupation_Adm-clerical',
 'occupation_Craft-repair',
 'occupation_Exec-managerial',
 'occupation_Farming-fishing',
 'occupation_Handlers-cleaners',
 'occupation_Machine-op-inspct',
 'occupation_Other-service',
 'occupation_Prof-specialty',
 'occupation_Protective-serv',
 'occupation_Sales',
 'occupation_Tech-support',
 'occupation_Transport-moving',
 'sex_Female',
 'sex_Male']

,index,age,education-num,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,education_10th,education_11th,education_12th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_HS-grad,education_Masters,education_Prof-school,education_Some-college,marital-status_Married-civ-spouse,marital-status_Not-married,occupation_?,occupation_Adm-clerical,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,sex_Female,sex_Male,education_1st-4th
0,3873,17,14,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,3625,23,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,3028,19,8,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,13814,30,9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,15398,60,13,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,6726,19,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
5096,1501,23,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
5097,6391,36,8,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
5098,7104,33,11,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


[1 0 0 ... 0 0 1]


0    3947
1    1153
Name: 1, dtype: int64

In [22]:
#予測ファイルの生成
sample_submit.to_csv('my_submission.csv', index=False, header=False)